In [4]:
import os 
from zhipuai import ZhipuAI
client=ZhipuAI()

In [6]:
tools=[
    {
        "type":"function",
        "function":{
            "name":"get_current_weather",
            "description":"获取指定城市的天气信息",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"城市，如：北京"
                    },
                    "locationCityCode":{
                        "type":"string",
                        "description":"城市编码,如：110000"
                    },
                    "unit":{
                        "type":"string",
                        "enum":["c","f"]
                    }
                },
                "required":["location","locationCityCode"]
            }
        }
    }
]

messages=[{"role":"user","content":"今天天津天气如何？"}]

completion=client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
completion.choices

[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8489581444274912209', function=Function(arguments='{"location":"天津","locationCityCode":"120000","unit":"c"}', name='get_current_weather'), type='function')]))]

In [7]:
import requests
import os
def get_current_weather(location:str,locationCityCode:str,unit:str):
    key=os.environ["tianqikey"]
    url=f"https://restapi.amap.com/v3/weather/weatherInfo?key={key}&city={locationCityCode}"
    print(url)
    response=requests.get(url)
    print(response.json())
    if response.status_code == 200:
        data=response.json()
        weather={
            "temperature":data["lives"][0]["temperature"],
            "description":data["lives"][0]["weather"]
        }
        return weather
    else:
        raise Exception(f"Failed retrive weatherL{response.status_code} ")


print(get_current_weather(location="北京",locationCityCode="110000",unit="c"))





https://restapi.amap.com/v3/weather/weatherInfo?key=225630f84b6a0f7626e97b3cda0d6e0e&city=110000
{'status': '1', 'count': '1', 'info': 'OK', 'infocode': '10000', 'lives': [{'province': '北京', 'city': '北京市', 'adcode': '110000', 'weather': '晴', 'temperature': '14', 'winddirection': '南', 'windpower': '≤3', 'humidity': '21', 'reporttime': '2024-03-18 17:33:50', 'temperature_float': '14.0', 'humidity_float': '21.0'}]}
{'temperature': '14', 'description': '晴'}


In [13]:
import json
def extract_function_and_execute(llm_output,messages):
    name=llm_output.choices[0].message.tool_calls[0].function.name
    params=json.loads(llm_output.choices[0].message.tool_calls[0].function.arguments)
    print(f'FunctionName:{name}')
    function_to_call=globals().get(name)
    if not function_to_call:
        raise ValueError(f"Function '{name}' not found")
    
    messages.append({
        "role":"tool",
        "content":str(function_to_call(**params))
    })
    return messages


messages=[
    {
        "role":"user",
        "content":"今天北京天气如何？"
    }
]

completion=client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
extract_function_and_execute(llm_output=completion,messages=messages)

final_anwser=client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tool_choice="auto"
)
final_anwser

FunctionName:get_current_weather
https://restapi.amap.com/v3/weather/weatherInfo?key=225630f84b6a0f7626e97b3cda0d6e0e&city=110000
{'status': '1', 'count': '1', 'info': 'OK', 'infocode': '10000', 'lives': [{'province': '北京', 'city': '北京市', 'adcode': '110000', 'weather': '晴', 'temperature': '14', 'winddirection': '西南', 'windpower': '≤3', 'humidity': '21', 'reporttime': '2024-03-18 18:03:26', 'temperature_float': '14.0', 'humidity_float': '21.0'}]}


Completion(model='glm-4', created=1710756353, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='今天北京的天气是晴天，温度大约在14摄氏度左右。请注意，这是一个根据我最后更新的信息生成的回答，实际天气情况可能会有所变化。如果你需要最新的天气信息，建议查看最近的天气预报。', role='assistant', tool_calls=None))], request_id='8489583230981392414', id='8489583230981392414', usage=CompletionUsage(prompt_tokens=24, completion_tokens=45, total_tokens=69))

In [14]:
tools_web_search=[
    {
        "type":"web_search",
        "web_search":{
            "eanble":False
        }
    }
]
messages=[{
    "role":"user",
    "content":"2023年100强独角兽企业有哪些?"
}]

completion_without_web=client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools_web_search
)
completion_without_web

Completion(model='glm-4', created=1710756558, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='根据提供的信息，2023年中国准独角兽企业100强的榜单中涉及了多个领域，包括人工智能、物流运输、先进制造、机器人、半导体等。具体的企业名单在摘要中并未完全列出，但以下是一些被提及的企业：\n\n1. 视比特机器人：位于长沙，专注于AI 3D视觉产品、工业智能软件及智慧工厂解决方案。\n2. 中储智运：具体信息不详，但从名字来看可能与智慧物流有关。\n3. Boreasa贝丰科技：具体信息不详。\n4. 蔚蓝科技：具体信息不详。\n5. 一知智能：具体信息不详。\n6. 知医科技：具体信息不详，可能与医疗健康领域有关。\n7. 大族机器人：可能专注于机器人技术。\n8. 微纳星空：可能与半导体或者精密仪器有关。\n9. 英诺赛科：具体信息不详。\n10. 云徙科技：具体信息不详。\n\n此外，还有以下提及的企业：\n\n- 商越科技：专注于提供采购数字化解决方案，服务于大中型企业。\n  \n榜单中还有其他企业，但未在摘要中提及。需要注意的是，这些企业被认为是“准独角兽”，意味着它们有潜力在未来短时间内成长为估值达到10亿美元或以上的独角兽企业。\n\n要获取完整的2023年中国准独角兽企业100强榜单，建议查阅互联网周刊、德本咨询和eNet研究院等机构发布的正式报告或相关官方发布信息。', role='assistant', tool_calls=None))], request_id='8489583780737239074', id='8489583780737239074', usage=CompletionUsage(prompt_tokens=1040, completion_tokens=300, total_tokens=1340))